In [ ]:
import json
import os
import tarfile

from web3 import Web3

data_directory = os.path.join("..", "..", "..", "data")

file_name = "displacement_bot_bytecode"
directory_path = os.path.join(data_directory, file_name)
compressed_path = os.path.join(data_directory, "{}.tar.xz".format(file_name))

if os.path.exists(compressed_path):
    with tarfile.open(compressed_path, "r:xz", encoding="utf-8") as compressed_file:
        compressed_file.extractall(data_directory)
elif not os.path.exists(directory_path):
    os.os.makedirs(directory_path)

#TODO fill in this values
server = Web3(Web3.HTTPProvider("<fill host port here"))

results_file_path = os.path.join(data_directory, "displacment_results.json")

if not os.path.exists(results_file_path):
    with tarfile.open(os.path.join(data_directory, "displacement_results.tar.xz"), "r:xz", encoding="utf-8") as compressed_file:
        compressed_file.extract("displacement_results.json", data_directory)

with open(results_file_path, "r", encoding="utf-8") as results_file:
    results = []
    line = results_file.readline().strip()
    while line != "":
        results.append(json.load(line))
        line  = results_file.readline().strip()

first_block_by_bots = {}

for result in results:
    if result["attacker_transaction"]["to"] == result["victim_transaction"]["to"]:
        continue
    attacker_transaction = result["attacker_transaction"]

    bot = attacker_transaction["to"]
    block = attacker_transaction['blockNumber']
    if bot not in first_block_by_bots:
        first_block_by_bots[bot] = block
    elif block < first_block_by_bots[bot]:
        first_block_by_bots[bot] = block

for bot, first_block in first_block_by_bots.items():
    code_file_path = os.path.join(directory_path, "{}-{}.bin".format(bot, first_block))

    if os.path.exists(code_file_path):
        continue

    bot = Web3.toChecksumAddress(bot)

    code = server.eth.getCode(bot, block_identifier=first_block)

    with open(code_file_path, "wb") as code_file:
        code_file.write(code)

    with tarfile.open(compressed_path, "w:xz") as compressed_file:
        compressed_file.add(directory_path, arcname=os.path.basenmae(file_name))